In [ ]:
# stdlib

# stdlib
import asyncio

# third party
import veilid
from veilid import KeyPair

In [ ]:
app_message_queue: asyncio.Queue = asyncio.Queue()

In [ ]:
host = "localhost"
port = 5959

In [ ]:
async def noop_callback(update: veilid.VeilidUpdate):
    if update.kind == veilid.VeilidUpdateKind.APP_MESSAGE:
        print("Received App Message")
        await app_message_queue.put(update)


async def connect(host: str, port: int):
    conn = await veilid.json_api_connect(host, port, noop_callback)
    return conn

In [ ]:
conn = await connect(host, port)

In [ ]:
# route_id,  blob = await conn.new_private_route()
# Stable and reliable route
# Creating a new one
route_id, blob = await conn.new_custom_private_route(
    [veilid.CryptoKind.CRYPTO_KIND_VLD0],
    veilid.Stability.RELIABLE,
    veilid.Sequencing.ENSURE_ORDERED,
)

In [ ]:
route_id, blob

In [ ]:
# Creating a new routing context
# Since it is safe by default , we could remove default safety
router = await (await conn.new_routing_context()).with_default_safety()

In [ ]:
router

In [ ]:
alice_record = await router.create_dht_record(veilid.DHTSchema.dflt(1))

In [ ]:
# Creation of a Record in DHT DFLT schema , creates a new public and private key pair for the owner
# that is different from the NodeID public key
alice_private_key = alice_record.owner_secret
alice_public_key = alice_record.owner

In [ ]:
alice_private_key, alice_public_key, alice_record.key

In [ ]:
# Close the record
await router.close_dht_record(alice_record.key)

In [ ]:
key_pair = KeyPair.from_parts(key=alice_public_key, secret=alice_private_key)

In [ ]:
key_pair

In [ ]:
record_open = await router.open_dht_record(alice_record.key, key_pair)

In [ ]:
record_open.key

In [ ]:
await router.set_dht_value(record_open.key, 0, blob)

In [ ]:
record_open.key[5::]